In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [1]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


In [2]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

In [3]:
q = 0
other_qs = list(range(translator.n_qubits))
other_qs.remove(q)
indds = [translator.cnots_index[str([q,k])] for k in other_qs]
cdb = pd.DataFrame([gate_template(i) for i in indds])

circuit_db = concatenate_dbs([z_layer_db(translator), cdb])

for q in range(1, translator.n_qubits):
    other_qs = list(range(translator.n_qubits))
    other_qs.remove(q)
    indds = [translator.cnots_index[str([q,k])] for k in other_qs]
    cdb = pd.DataFrame([gate_template(i) for i in indds])
    circuit_db = concatenate_dbs([circuit_db ,cdb])

c, circuit_db = translator.give_circuit(circuit_db)

db1 = concatenate_dbs([x_layer_db(translator)]*3)
#db2 = concatenate_dbs([z_layer_db(translator)]*3)
#db3 = concatenate_dbs([x_layer_db(translator)]*3)
#db4 = concatenate_dbs([z_layer_db(translator)]*3)

circuit_db = concatenate_dbs([circuit_db, db1])
c, circuit_db = translator.give_circuit(circuit_db)

cdb = []
for ind, qubits in list(translator.indexed_cnots.items()):
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
c6_db = pd.DataFrame(cdb)
circuit_db = concatenate_dbs([circuit_db, c6_db])

c, circuit_db = translator.give_circuit(circuit_db)

In [4]:
c

┌──┐                                                                                                                                                     ┌──┐
(0, 0): ───Rz(th_0)───@───@───@───X─────────X────────────X───Rx(th_4)───Rx(th_8)───Rx(th_12)───────────────@───────────Rz(th_16)───@───Rz(th_18)───@───Rz(th_20)───X───Rx(th_23)─────────────────────X────Rx(th_29)───────────────────────────────────X───Rx(th_35)───────────────────────────────────
                      │   │   │   │         │            │                                                 │                       │               │               │                                 │                                                │
(0, 1): ───Rz(th_1)───X───┼───┼───@───@────@┼────X───────┼───X──────────Rx(th_5)───Rx(th_9)────Rx(th_13)───X───────────Rx(th_17)───┼───────────────┼───────────────@───Rz(th_22)───@───Rz(th_24)────@┼────Rz(th_26)───X───Rx(th_31)───────────────────┼───────────────X───Rx(th_37)───────────────────
                          │   │       │    ││    │       │   │                                                                     │               │                               │                ││                │                               │               │
(0, 2): ───Rz(th_2)───────X───┼───────X────┼@────@───@───┼───┼──────────X──────────Rx(th_6)────Rx(th_10)───Rx(th_14)───────────────X───Rx(th_19)───┼───────────────────────────────X───Rx(th_25)────┼@────Rz(th_28)───@───Rz(th_30)───@───Rz(th_32)───┼───────────────┼───────────────X───Rx(th_39)───
                              │            │         │   │   │          │                                                                          │                                                │                                 │               │               │               │
(0, 3): ───Rz(th_3)───────────X────────────X─────────X───@───@──────────@──────────Rx(th_7)────Rx(th_11)───Rx(th_15)───────────────────────────────X───Rx(th_21)────────────────────────────────────X─────Rx(th_27)───────────────────X───Rx(th_33)───@───Rz(th_34)───@───Rz(th_36)───@───Rz(th_38)───
                                          └──┘                                                                                                                                                     └──┘

In [5]:
simplifier = Simplifier(translator, apply_relatives_to_first = True)
simplifier.absolute_rules = [simplifier.rule_4, simplifier.rule_5, simplifier.rule_6]
simplifier.relative_rules = [simplifier.rule_1, simplifier.rule_2]
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 

AttributeError: ojo <bound method Simplifier.rule_1 of <utilities.simplifier.Simplifier object at 0x7eff1ca094a8>>

In [ ]:
mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(c)
print()
print(ss)

THE PROBLEM IS RULE 6 !!! 
come on!